In [1]:
import pandas as pd
import numpy as np
from pyvi import ViTokenizer, ViPosTagger, ViUtils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from collections import defaultdict

### Nhận tệp dữ liệu cần train

In [4]:
data = pd.read_excel('C:\\Users\\CYBER\\Desktop\\AI_PROJECT_NEW\\data\\processed_medical_data_AI.xlsx')

# Kiểm tra và xử lý giá trị null
data['Van_de_benh_nhan'].fillna('', inplace=True)
data['Benh'].fillna('', inplace=True)
data['Don_thuoc'].fillna('', inplace=True)

#loại bỏ dữ liệu bị trùng
data.drop_duplicates(inplace=True)
len(data)

C:\Users\CYBER\AppData\Local\Temp\ipykernel_9204\3830935338.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Van_de_benh_nhan'].fillna('', inplace=True)
C:\Users\CYBER\AppData\Local\Temp\ipykernel_9204\3830935338.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

4692

### Xử lý chuỗi input đầu vào bằng cách sử dụng thư viện PyVi về dạng vector

In [5]:
# Sử dụng PyVi để tokenize văn bản tiếng Việt
data['Tokenized_Problem'] = data['Van_de_benh_nhan'].apply(lambda x: ViTokenizer.tokenize(x))

In [6]:
# Tạo từ điển từ vựng sử dụng defaultdict
vocab = defaultdict(lambda: len(vocab) + 1)
for text in data['Tokenized_Problem']:
    for word in text.split():
        _ = vocab[word]  # Thêm từ vào từ điển nếu chưa có

# Chuyển đổi văn bản thành chuỗi số và pad sequences
sequences = data['Tokenized_Problem'].apply(lambda x: [vocab[word] for word in x.split() if word in vocab])
padded_sequences = pad_sequences(sequences, maxlen=100)

In [7]:
# Mã hóa nhãn đầu ra
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_labels = label_encoder_disease.fit_transform(data['Benh'])
prescription_labels = label_encoder_prescription.fit_transform(data['Don_thuoc'])

disease_labels_categorical = to_categorical(disease_labels)
prescription_labels_categorical = to_categorical(prescription_labels)

# print(padded_sequences[:5])
# print(disease_labels_categorical[:5])
# print(prescription_labels_categorical[:5])

In [ ]:
from tensorflow.keras.optimizers import Adam
input_layer = Input(shape=(100,)) #Kích thước đầu vào là 100
embedding_layer = Embedding(input_dim=len(vocab) + 1, output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding_layer)

disease_output = Dense(len(label_encoder_disease.classes_), activation='softmax', name='disease_output')(lstm_layer)
prescription_output = Dense(len(label_encoder_prescription.classes_), activation='softmax', name='prescription_output')(lstm_layer)

model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])
optimizer = Adam(learning_rate=0.01)
model.compile(
    loss={'disease_output': 'categorical_crossentropy',
          'prescription_output': 'categorical_crossentropy'},
    optimizer=optimizer,
    # optimizer='RMSprop',
    
    metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 64)   │     98,560 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     33,024 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disease_output      │ (None, 889)       │     57,785 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prescription_output │ (None, 2963)      │    192,595 │ lstm[0][0]        │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 381,964 (1.46 MB)

 Trainable params: 381,964 (1.46 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(padded_sequences, {'disease_output': disease_labels_categorical, 'prescription_output': prescription_labels_categorical}, epochs=60, batch_size=64)

Epoch 1/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - disease_output_accuracy: 0.0174 - disease_output_loss: 6.2517 - loss: 14.3476 - prescription_output_accuracy: 0.0077 - prescription_output_loss: 8.0957
Epoch 2/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - disease_output_accuracy: 0.0337 - disease_output_loss: 5.7181 - loss: 13.3215 - prescription_output_accuracy: 0.0104 - prescription_output_loss: 7.6034
Epoch 3/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - disease_output_accuracy: 0.0661 - disease_output_loss: 5.3217 - loss: 12.6003 - prescription_output_accuracy: 0.0222 - prescription_output_loss: 7.2787
Epoch 4/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - disease_output_accuracy: 0.1001 - disease_output_loss: 4.9144 - loss: 11.7576 - prescription_output_accuracy: 0.0384 - prescription_output_loss: 6.8434
Epoch 5/100
74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - disease_output_accuracy: 0.1398 - disease_output_loss: 4.3672 - loss: 10.6121 - prescription_output_accuracy: 0.0485 - pre

### Xây dựng hàm dự đoán bệnh và đơn thuốc từ mô tả ban đầu của bệnh nhân

In [18]:
def make_prediction(input_text):
    tokenized_text = ViTokenizer.tokenize(input_text)
    sequence = [vocab[word] for word in tokenized_text.split() if word in vocab] #chuyển đổi văn bản thành chuối số
    padded_sequence = pad_sequences([sequence], maxlen=100)
    prediction = model.predict(padded_sequence)
    
    disease_prediction = label_encoder_disease.inverse_transform(np.argmax(prediction[0], axis=1))
    prescription_prediction = label_encoder_prescription.inverse_transform(np.argmax(prediction[1], axis=1))
    
    return disease_prediction[0], prescription_prediction[0]

In [19]:
def main():
    while True:
        input_text = input("Nhập mô tả bệnh (hoặc gõ 'exit' để thoát): ")
        if input_text.lower() == 'exit':
            break
        disease, prescription = make_prediction(input_text)
        print(f"Vấn đề của bệnh nhân: {input_text}")
        print(f"Bệnh dự đoán: {disease}")
        print(f"Đơn thuốc dự đoán: {prescription}")

main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
Vấn đề của bệnh nhân: tôi thường xuyên bị khó tiêu, ợ chua và đầy hơi
Bệnh dự đoán: Ung thư phổi
Đơn thuốc dự đoán: Hóa trị, liệu pháp thuốc nhắm mục tiêu.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Vấn đề của bệnh nhân: tôi thường xuyên bị khó tiêu, ợ chua và đầy hơi
Bệnh dự đoán: Ung thư phổi
Đơn thuốc dự đoán: Hóa trị, liệu pháp thuốc nhắm mục tiêu.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Vấn đề của bệnh nhân: 
Bệnh dự đoán: 
Đơn thuốc dự đoán: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Vấn đề của bệnh nhân: đau bụng
Bệnh dự đoán: 
Đơn thuốc dự đoán: 


In [5]:
import joblib

try:
    vocab = joblib.load("C:\\Users\\tontide1\\my-data\\AI\\AI_PROJECT_NEW (1)\\AI_PROJECT_NEW\\data\\tokenizer_data\\vocab.pkl")
    print("vocab loaded successfully:", vocab)
except EOFError:
    print("EOFError: vocab.pkl file may be incomplete or corrupted.")
except FileNotFoundError:
    print("FileNotFoundError: vocab.pkl file not found in the specified directory.")


vocab loaded successfully: {'Thường_xuyên': 1, 'bị': 2, 'đau': 3, 'nửa': 4, 'đầu': 5, 'dữ_dội': 6, ',': 7, 'nhạy_cảm': 8, 'với': 9, 'ánh_sáng': 10, 'và': 11, 'âm_thanh': 12, '.': 13, 'Tăng': 14, 'cân': 15, 'đột_ngột': 16, 'cảm_thấy': 17, 'lạnh': 18, 'đặc_biệt': 19, 'là': 20, 'ở': 21, 'tay': 22, 'chân': 23, 'Sốt_cao': 24, 'họng': 25, 'hạch': 26, 'bạch_huyết': 27, 'sưng': 28, 'rất': 29, 'yếu': 30, 'Khát': 31, 'nước': 32, 'nhiều': 33, 'đi_tiểu': 34, 'thường_xuyên': 35, 'khô': 36, 'miệng': 37, 'dai_dẳng': 38, 'Đau': 39, 'ngực': 40, 'đau_nhói': 41, 'tăng': 42, 'lên': 43, 'khi': 44, 'nằm_xuống': 45, 'hoặc': 46, 'hít': 47, 'thở': 48, 'sâu': 49, 'Giảm': 50, 'không': 51, 'rõ': 52, 'nguyên_nhân': 53, 'đổ': 54, 'mồ_hôi': 55, 'đêm': 56, 'sốt': 57, 'bụng': 58, 'lan': 59, 'ra': 60, 'sau': 61, 'lưng': 62, 'buồn_nôn': 63, 'nhưng': 64, 'nôn': 65, 'Sưng': 66, 'cứng': 67, 'khớp': 68, 'vào': 69, 'buổi': 70, 'sáng': 71, 'ngón': 72, 'cổ_tay': 73, 'Các': 74, 'mụn': 75, 'đau_đớn': 76, 'tái_phát': 77, 'quanh':